In [13]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
%matplotlib
###########assign memory##########
###########delete this part if your tensorflow was based on CPU##########
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('database.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'BET Surface Area cm2/g',
                      'Micropore ratio',
                      'Mesopore ratio',
                      'Macropore ratio',
                      'Membrane Thickness',
                      'I/C Ratio(ionomer/catalyst)',
                      'Anodic Platinum Loading Amount mgPt cm-2',
                      'Anodic catalyst type x wt% Pt/C',
                      'Hot Press Temperature ℃',
                      'Hot Press Time min',
                      'Hot Press Pressure Mpa',
                      'Humidity %',
                      'Operating Temperature ℃',
                      'Flowing rate of H2 ml min-1 ',
                      'Flowing rate of O2 ml min-1 ',
                      'Back Pressure Mpa',
                      'Cathodic Loading Amount mg cm-2',
                      'Co Cotent wt/%',
                      'Fe Cotent wt/%(ICP for metal)',
                      'N Content at/%',
                      'O Content at/%',
                      'S Content at/%',
                      'Pyridinic N+metal N Content ratio at/%',
                      'Pyrrolic N Content ratio at/%',
                      'Graphitic N Content ratio at/%',
                      'Oxidized N Content ratio at/%',#26features
                      'Open Circuit Voltage V',
                      'Voltage 100mA cm0',
                      'Voltage 200mA cm0',
                      'Voltage 300mA cm0',
                      'Voltage 400mA cm0',
                      'Voltage 500mA cm0',
                      'Voltage 600mA cm0',
                      'Voltage 700mA cm0',
                      'Voltage 800mA cm0',
                      'Voltage 900mA cm0',
                      'Voltage 1000mA cm0',
                      'Voltage 1100mA cm0',
                      'Voltage 1200mA cm0',
                      'Voltage 1300mA cm0',
                      'Voltage 1400mA cm0',
                      'Voltage 1500mA cm0',
                      'Voltage 1600mA cm0',
                      'Voltage 1700mA cm0',
                      'Voltage 1800mA cm0',
                      'Voltage 1900mA cm0',
                      'Voltage 2000mA cm0',
                      'Voltage 2100mA cm0',
                      'Voltage 2200mA cm0',
                      'Voltage 2300mA cm0',
                      'Voltage 2400mA cm0',
                      'Voltage 2500mA cm0',
                      'Voltage 2600mA cm0',
                      'Voltage 2700mA cm0',
                      'Voltage 2800mA cm0',
                      'Maximum Power Density mW cm-2 ']]#55th row,30output results
###########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
raw_power=data.iloc[:,26:]
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)#即简单实现标准化
###########train test splitting##########
param=standardized_data.iloc[:,0:26]
power=standardized_data.iloc[:,55]
X=param.values.astype(np.float32)
y=power.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=751)
###########search neuron network hyperparmeter space##########
dropout_list= np.arange(0, 1, 0.1)
for neurons in range(50,750,25):
    for activation in ['relu','tanh','sigmoid','softsign','hard_sigmoid','softplus']:
        for regularizer_term in [0.0001,0.0005,0.0008,0.001,0.002,0.005,0.0008,0.01,0.05,0.1]:
            for dropout in dropout_list:
                for epochs_number in range(200,1000,200):
                    for batch_size_number in range (20,200,20):
                        for learning_rate_search in [0.001,0.002,0.005,0.01,0.02,0.05]:
                            ###########implementing hyperparameters##########
                            neurons1=neurons
                            activation1=activation
                            regularizer=keras.regularizers.l2(regularizer_term)
                            dropout_rate=dropout
                            ###########keras ANN model construction##########
                            model = Sequential() 
                            model.add(Dense(neurons1, input_dim=26, kernel_initializer='random_normal',
                                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate))
                            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate)) 
                            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate))
                            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate)) 
                            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate)) 
                            model.add(Dense(1, input_dim=neurons1, activation='linear'))
                            adam=optimizers.Adam(lr=learning_rate_search)
                            model.compile(loss='mse', optimizer=adam) 
                            #print('Training ------------')
                            ###########train the model with the training set##########
                            ###########testset has been remove before##########
                            model.fit(X_train, y_train,verbose=0, epochs=epochs_number, batch_size=batch_size_number,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
                            loss= model.evaluate(X_test, y_test)
                            predict_ann= model.predict(X_test)
                            train_ann= model.predict(X_train)
                            ###########result output##########
                            x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[55]+np.mean(data,axis=0)[55]
                            y_real_maximum_power=y_test*np.std(data,axis=0)[55]+np.mean(data,axis=0)[55]
                            x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
                            x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
                            y_real_maximum_power_series=pd.Series(y_real_maximum_power)
                            corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
                            rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
                            print('ANN,R2',corr_ann,'RMSE',rmse_val)
                            if 0.96<corr_ann<1:
                                break
                            else:
                                K.clear_session()
                        else:continue
                        break
                    else:continue
                    break
                else:continue
                break
            else:continue
            break
        else:continue
        break
    else:continue
    break
###########print best hyperparameter##########
print(learning_rate_search)
print(neurons)
print(activation)
print(regularizer_term)
print(dropout)
print(epochs_number)
print(batch_size_number)
print('ANN,R2',corr_ann)
print('ANN,RMSE',rmse_val)
print(y_real_maximum_power)
###########visualization##########
x_y_x=np.arange(0,1300,100)
x_y_y=np.arange(0,1300,100)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_prediction_maximum_power_ann,y_real_maximum_power,color='red',label='Artificial Neural Network')
plt.legend()
ax.plot(x_y_x,x_y_y)
plt.xlabel(u"Predicted_Maximum_Power mW cm^-2")
plt.ylabel(u"Real_Maximum_Power mW cm^-2")
plt.show()



Using matplotlib backend: Qt5Agg
21/21 [==============================] - 0s 0us/step
ANN,R2 0.933 RMSE 83.93146
21/21 [==============================] - 0s 0us/step
ANN,R2 0.9068 RMSE 90.64683
21/21 [==============================] - 0s 0us/step
ANN,R2 0.9033 RMSE 91.90589
21/21 [==============================] - 0s 0us/step
ANN,R2 0.9414 RMSE 83.18991
21/21 [==============================] - 0s 0us/step
ANN,R2 0.8989 RMSE 92.49124
21/21 [==============================] - 0s 0us/step
ANN,R2 0.7646 RMSE 148.37468
21/21 [==============================] - 0s 47us/step
ANN,R2 0.9226 RMSE 85.33896
21/21 [==============================] - 0s 0us/step
ANN,R2 0.9621 RMSE 58.502304
0.002
50
relu
0.0001
0.0
200
40
ANN,R2 0.9621
ANN,RMSE 58.502304
[750.  292.5 450.  940.  810.  680.  700.  450.  430.  365.  850.  300.
 650.  506.  360.  320.  530.  870.  810.  320.  530. ]
